In [ ]:
# no lstm # come from v3
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Conv1D, Activation, MaxPooling1D, Flatten, Attention, Input
import numpy as np
import pickle
from tensorflow.keras.layers.experimental import preprocessing


pickle_in = open("X_train_full.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train_full.pickle","rb")
y_train = pickle.load(pickle_in)


pickle_in = open("X_test_full.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("y_test_full.pickle","rb")
y_test = pickle.load(pickle_in)

# layer = tf.keras.layers.LayerNormalization(axis=[2])
# output = layer(X_test)

In [ ]:
count = 0
sample = X_train.shape[0]
rows = X_train.shape[1]
columns = X_train.shape[2]
for i in range(0, sample):
    for x in range(0,rows):
        for y in range(0,columns):
            if(X_train[i][x][y] in {'NaN','infinity','nan','Infinity'}):
                count+=1
                print("Deleted Rows: "+str(count))

In [ ]:
#Normalized Data
'''
X_train_normalized = ((X_train-np.min(X_train))/(np.max(X_train)-np.min(X_train)))
X_test_normalized = ((X_test-np.min(X_test))/(np.max(X_test)-np.min(X_test)))


layer = preprocessing.Normalization()
layer.adapt(X_train)
X_train_normalized = layer(X_train)

layer = preprocessing.Normalization()
layer.adapt(X_test)
X_test_normalized = layer(X_test)
'''

In [ ]:
# v = tf.signal.dct(X_train, type=2, n=None, axis=-1, norm=None, name=None)

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Input, Flatten, Activation, RepeatVector, Permute, Multiply, Lambda, Concatenate
import tensorflow.keras.backend as K
from tensorflow import keras 

# input layer
EEG = Input(shape=(X_train.shape[1:]))

# first feature extractor

norm1 = tf.keras.layers.LayerNormalization(axis=2)(EEG)

conv1 = Conv1D(64, (3), input_shape=(X_train.shape[1:]))(norm1)
relu1 = Activation('relu')(conv1)

conv2 = Conv1D(64, (2))(relu1)
relu2 = Activation('relu')(conv2)
pool1 = MaxPooling1D(pool_size=(2))(relu2)

conv3 = Conv1D(64, (2))(pool1)
relu3 = Activation('relu')(conv2)
pool2 = MaxPooling1D(pool_size=(2))(relu3)

flat1 = Flatten()(pool2)

# second feature extractor
DCT1 = tf.signal.dct(EEG, type=2, n=None, axis=-1, norm=None, name=None)
norm2= tf.keras.layers.LayerNormalization(axis=2)(DCT1)
#activations = LSTM(128, input_shape=(X_train.shape[1:]), activation='relu', return_sequences=True)(norm2)

# compute importance for each step
attention = Dense(1, activation='tanh')(norm2)
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(128)(attention)
attention = Permute([2, 1])(attention)

sent_representation = Multiply()([norm2, attention])
sent_representation = Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation)

flat2 = Flatten()(sent_representation)

# merge feature extractors
merge = Concatenate()([flat1, flat2])

dence1 = Dense(512)(merge)

dence2= Dense(8)(dence1)
softmax1 = Activation('softmax')(dence2)
model = keras.Model(inputs=EEG, outputs=softmax1)


In [ ]:
model.summary()

In [ ]:
y_train = keras.utils.to_categorical(y_train, 8)
y_test = keras.utils.to_categorical(y_test, 8)

In [ ]:
np.array(y_train).shape

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

NAME = "conv1D_DCT_full_v1"
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))


opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

# checkpoint
filepath="conv1D_DCT_full_v1.weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.fit(X_train,
          np.array(y_train),
          epochs=20,
          validation_data=(X_test, np.array(y_test)),
         callbacks=callbacks_list)

In [ ]:
prediction = model.predict([X_test[0].reshape(-1,440,128)])
prediction

In [ ]:
#model.save('conv1D_46_46_64_512_2.model')

In [ ]:
CATEGORIES = ["blue_sky", "green_color"]

In [ ]:
# model = tf.keras.models.load_model("conv1D_46_46_64_512_2.model")

prediction = model.predict([X_test[0].reshape(-1,440,128)])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])